In [ ]:
import numpy as np

In [ ]:
class LinearDiscriminantAnalysis:
  def __init__(self,components):
    self.components = components
    self.ld = None
  
  def fit(self,X,y):
    features = X.shape[1]
    sep_within = np.zeros((features,features))
    sep_between = np.zeros((features,features))
    distinct_labels = np.unique(y)
    total_mean = np.mean(X,axis = 0)
    
    for i in distinct_labels:
      input = X[y==i]
      mean = np.mean(input,axis = 0)
      sep_within += np.dot((input-mean).T,(input-mean))
      number_of_sample = input.shape[0]
      mean_difference = (mean-total_mean).reshape(features,1)
      sep_between += input.shape[0]*np.dot(mean_difference,mean_difference.T)

    A = np.dot(np.linalg.inv(sep_within),sep_between)
    eigenvalues , eigenvectors = np.linalg.eig(A)
    eigenvectors = eigenvectors.T
    i = np.argsort(abs(eigenvalues))[::-1]
    eigenvector = eigenvectors[i]
    eigenvalues = eigenvalues[i]
    self.ld = eigenvectors[:self.components] 

  def predict(self,X):
    return np.dot(X,self.ld.T)

class NaiveBayes():
  def fit(self,X,y):
    samples,features = X.shape
    self.labels = np.unique(y)
    self.mean = np.zeros((len(self.labels),features),dtype=np.float64)
    self.variance = np.zeros((len(self.labels),features),dtype=np.float64)
    self.prior_prob = np.zeros(len(self.labels),dtype=np.float64)
    # print(self.mean)
    for x in self.labels:
      inp = X[y==x]
      self.mean[x,:] = inp.mean(axis = 0)
      self.variance[x,:] = inp.var(axis = 0)
      self.prior_prob[x] = inp.shape[0]/float(samples)

  def posterior(self,X):
    posterior_probability = []
    for i,x in enumerate(self.labels):
        prior = np.log(self.prior_prob[i])
        posterior = np.sum(np.log(self.Gaussian(i, X)))
        posterior = prior + posterior
        posterior_probability.append(posterior)
    return self.labels[np.argmax(posterior_probability)]
  
  def Gaussian(self, class_i, x):
    mean = self.mean[class_i]
    # print(mean)
    var = self.variance[class_i]
    return (np.exp(-((x - mean) ** 2) / (2 * var))) / (np.sqrt(2 * np.pi * var))

  def predict(self,X):
    return np.array([self.posterior(x) for x in X])

class LogisticRegression():
  def fit(self,X,y,alpha = 0.01,steps = 100):
    samples,features = X.shape
    self.weights = np.zeros(features+1)
    data = np.hstack((X,np.ones((X.shape[0],1))))
    for i in range(steps):
      predicted = 1/(1+np.exp(-np.dot(data,self.weights)))
      dw = np.dot(np.dot(data.T,(predicted-y)),np.dot(predicted,(1-predicted)))
      self.weights -= alpha*dw
  
  def predict(self,X):
    target = 1/(1+np.exp(-np.dot(np.hstack((X,np.ones((X.shape[0],1)))),self.weights)))
    target = np.where(target>0.5,1,0)
    return target

def mean_squared(target,predicted):
  return 1-np.square(np.subtract(target,predicted)).mean()